# 438. Find All Anagrams in a String
Given two strings, `s` and `p`, return an array of all of the start indices of `p`'s anagrams in `s`.
Return the answer in any order.

An anagram is a word of phrase formed by rearranging the letters of a different word
or phrase, typically using all of the original letters exactly once.

**Start: 12:04**

**End: NA**

Thought about this problem on and off for the majority of the day inbetween microscopy sessions.

Did not record a true end time, as I produced several solutions that were too slow and then dug into a more effective solution.


### Planning
Ok - I'm thinking that to do this I will iterate over `s` once. 

At each iteration step, I'll check to see if the current string is in the sublist.

I'll also keep a running copy of `p`. When iterating over `s`, I'll check to see if the
individual elements are in `p`. If they are, I'll remove them from `p` and subiterate to the
next index of `s`. If an element is not in `p`, I'll break the subloop, recopy `p`, and
continue on.

My other line of thought is this:
If when we get to a breaking point of `s` we find that that element is not in `p`:
1. If that element is not in the original copy of `p`, then we start from that index.
2. If that element is in the original copy of `p`, we should start from the parent iteration
index.

In [18]:
def find_anagrams(s, p):
    anagram_starts = []
    
    p_copy = list(p)
    i = 0
    while i < len(s) - len(p) + 1:  # iterate over s until there is no more room for p
        # make sure p_copy is the same as p
        if len(p_copy) != len(p):
            p_copy = list(p)

        # check to see if the letter is in p_copy
        j = i
        while len(p_copy):
            letter = s[j]
            
            # if the letter is in p_copy, remove it from the p_copy list
            if letter in p_copy:  
                p_copy.pop(p_copy.index(letter)) 
                j += 1
                if not len(p_copy):
                    anagram_starts.append(i)  # if p_copy is empty, anagram found
                    break
            # if the letter is not in p_copy, get out of the loop
            else:  
                # if the letter is in p, we need to keep checking at the next i index
                # if it's not, we can skip past the previously examined positions
                if letter not in p:  
                    i = j
                break
            
        i += 1
        
    return anagram_starts

s = "abab"
p = "ab"
find_anagrams(s, p)

[0, 1, 2]

I had a whole series of text typed out explaining my path of logical progression, but accidentally deleted it.

Here is where I'm at currently:

In [24]:
from random import random
def find_anagrams(s, p):
    anagram_indices = []
    
    # generate hashes
    hash_map = {i: int(i * 102 / 4) for i in range(26)}
    def hash_string(c):
        # generate the hash map
        hashed_string = [
            hash_map[ord(letter) - 97] for letter in c
        ]
        return hashed_string
    
    # hash the lists
    p, s = hash_string(p), hash_string(s)
    print(p, s, " "* 25, end="\r")
    p_hash = sum(p)
    
    # get the set of letters in p for rapid determination of whether we need to hash
    p_set = set(p)
    for i in range(len(s)-len(p)+1):
        current_letter = s[i]
        if current_letter in p_set:  # hash the letters if the current letter is in p
            current_hash = sum(s[i:i+len(p)])
            if current_hash == p_hash:
                anagram_indices.append(i)
    
    return anagram_indices

s = "cbaebabacd"
p = "abc"

find_anagrams(s, p)

[0, 6]

The solution above is not perfect. Sometimes the hashes aren't produced correctly
and don't generate appropriate unique values.

Because I've solved this problem with two different approaches, I decided to look to see
how others solved it.

I noticed that instead of truly hashing each value, some people create keymaps with
that match to see how many values of each letter occur in `p` and then iterate through `s`
to find anagrams.

Cool - I've found the ongoing and active algorithm that most people are using to solve this
problem.

Rather than creating a hash of each individual anagram, letter, or brute forcing, instead
the solution is to use a sliding window that keeps track of the number of letters within each
sliding window.

By using this approach, at each stride, only the count of the letter that just entered the window
and the count of the letter that just left the window need to be updated.

I'm going to write it in a less efficient manner first so it's obvious how/why the solution works.

In [72]:
from collections import defaultdict


def find_anagrams_window_slow(s, p):
    anagram_indices = []
    
    # create the map of the number of letters in p
    p_map = defaultdict(int)
    for letter in p:
        p_map[letter] += 1
        
    # now slide over each window of s with window size len(p) and stride of 1
    for i in range(0, len(s)-len(p)+1):
        window_counts = defaultdict(int)
        for letter in s[i:i+len(p)]:
            window_counts[letter] += 1
        
        # if the letter frequencies match, add the index
        if window_counts == p_map:
            anagram_indices.append(i)
    
    return anagram_indices

s = "cbaebabacd"
p = "abc"

find_anagrams_window_slow(s, p)

[0, 6]

The approach above is accurate but very slow, as we have to create a new hash map
for every single window.

In the next approach, rather than creating a new map for each sliding window, we can
have a single window_map. This map will only have the values that the p_map has.

The window_map will only update letters that have just entered and then just left
the map. It will start its stride with its right edge at the beginning of `s`.

If the maps match one another at the end of an iteration, append the index of the beginning
of the stride.

We will iterate one value behind the window.

In [73]:
def find_anagrams_window_two_maps(s, p):
    if len(s) < len(p):
        return []
    
    anagram_indices = []
    p_map = defaultdict(int)
    for letter in p:
        p_map[letter] += 1
        
    window_map = {letter: 0 for letter in p_map.keys()}
    
    # start right edge of window at 0
    for i in range(-len(p), len(s)-len(p)):
        if i > -1 and s[i] in p_map:  # remove old letters from the window_map
            window_map[s[i]] -= 1
            
        if s[i+len(p)] in p_map:  # add new letters to he window map
            window_map[s[i+len(p)]] += 1
        
        if window_map == p_map:  
            anagram_indices.append(i + 1)
    
    return anagram_indices

s = "cbaebabacd"
p = "abc"

find_anagrams_window_two_maps(s, p)

[0, 6]

Now finally, rather than using two maps to keep track of the window values and the
p values, we can instead use a single map that acts as a difference of the two maps.

This is because when we are checking if window_map == p_map, in reality we are just
performing a subtraction to see if there is a zero sum difference between the two maps after the iteration's value updates.

`p_map - window_map = difference`

`p_map = {'a': 1, 'b': 1, 'c': 1}`

`window_map = {'a': 0, 'b': 0, 'c': 1}`

`difference = {'a': 1, 'b': 1, 'c': 0}`

Therefore, we can just update the `difference` of these two, since the starting `difference == p_map`.

As such, if we were to apply a subtraction update to the `window_map`, we would add it to the `difference`, aka `p_map`. Vice versa for addition updates. See below for an example.


0:
Window_right = c, window_left=NA --- `p_map = {'a': 1, 'b': 1, 'c': 0}` (subtract 1 from p_map['c'] aka add 1 to window_map['c'])
1:
Window_right = b, window_left=NA --- `p_map = {'a': 1, 'b': 0, 'c': 0}` (subtract 1 from p_map['b'] aka add 1 to window_map['b'])
2:
Window_right = a, window_left=NA --- `p_map = {'a': 0, 'b': 0, 'c': 0}` (subtract 1 from p_map['a'] aka add 1 to window_map['a'])
3:
Window_right = NA, window_left=c --- `p_map = {'a': 0, 'b': 0, 'c': 1}` (add 1 to p_map['c'] aka subtract 1 from window_map['c'])

In [76]:
from collections import defaultdict


def find_anagrams(s, p):
    if len(s) < len(p):
        return []
    
    anagram_indices = []
    
    # Create the keymap that contains the occurrence of each letter in p
    p_map = defaultdict(int)
    for letter in p:
        p_map[letter] += 1
    p_map_set = set(p_map.keys())
    
    for i in range(-len(p), len(s)-len(p)):
        if i > -1 and s[i] in p_map_set:  
            p_map[s[i]] += 1
        
        new_letter = s[i+len(p)]
        if new_letter in p_map_set:
            p_map[new_letter] -= 1
            
        if not any(p_map.values()):
            anagram_indices.append(i+1)
    
    return anagram_indices


s = "cbaebabacd"
p = "abc"

find_anagrams(s, p)

[0, 6]

### Afterthoughts
**Stats:**
Runtime
262 ms
Beats
44.29%
Memory
15.1 MB
Beats
7.8%

The stats aren't great, but I'm really happy with my ability to break down and understand the algorithmic
approaches that others took. I like the sliding window approach quite a bit.

# 424. Longest Repeating Character Replacement
Given a substring `s` and an integer `k`, choose any character of the string `s` and replace
it with another uppercase character. This operation can be performed `k` times.

Return the length of the longest substring with a sequence of a single value.

E.g., 

`s = "ABAB"`

`k = 2`

`output = 4` since `"AAAA"` or `"BBBB"`

**Start: 18:28**

**End: 19:12**

### Planning
My first intuition was to perform a dif of `s[:1]-s[1:]` to find where the non-zero
indices were. We could then replace `k` of those nonzero indices to find the longest
sequences of zeros. 

This approach would end up being a bit cumbersome, because we'd have to
iterate over the unique non-zero indices, replace those values with zeros (possibly multiple times), and then find the longest sequence of zeros within each iteration and possibly sub iteration.

Instead, I will use a sliding window approach. 

This sliding window will increase in size at rate of 1 for each tick if the next value
matches the left edge value. If it doesn't, it will change the new right edge value to the left edge value
and grow by one. This will continue until there are no `k` left or until the right edge meets the string edge.

The ongoing longest string will be recorded. 

`while right_edge < len(s)`:
1. Create sliding window with left and right edges at [0]
2. Increase the right edge index by 1.
3. If the new value matches the left edge value, return to 2, else go to 4.
4. If the new value doesn't match and we have any swaps left, swap the value, return to 2. Record the first occurrence of a non-left value.
If no `k` are left, go to `5`.
5. If the new value doesn't match and we have no `k` left, move left edge to the first non-left value, record the length of the string.


In [139]:
def character_replacement(s, k) -> int:
    if len(s) == 1:
        return 1
    
    left_edge, right_edge = 0, 0
    left_letter = s[0]
    max_length = 1
    remaining_k = k
    new_left = None
    
    while right_edge < len(s):
        current_string = s[left_edge:right_edge]
        if left_letter == s[right_edge]:  # check leading edge with trailing
            right_edge += 1
            
        elif remaining_k > 0:
            # we can change the trailing, but only once if a separation of 2 and the
            # middle letter is the same as the right
            if right_edge - left_edge == 2:
                if s[right_edge-1] == s[right_edge]:
                    left_letter = s[right_edge]
                    if remaining_k != k:  # add one back to rem_k if this is the case
                        remaining_k += 1
            new_left = right_edge if not new_left else new_left
            right_edge += 1
            remaining_k -= 1
            
        else:
            max_length = max(max_length, len(s[left_edge:right_edge]))
            remaining_k = k
            left_edge = new_left if new_left else right_edge
            left_letter = s[left_edge]
            right_edge = left_edge + 1
            new_left = None
            
    # check the final cases
    max_length = max(max_length, len(s[left_edge:right_edge]))
    return max_length

s = "ABABA"
k = 1
character_replacement(s, k)

3

With my first implementation, I passed 26/37 of test cases.

The issue with my algorithm arises with the following test case:

`s = "ABBB"`

`k = 2`

The length here should be `4`, but instead my algorithm only finds `3`. An obvious and
simple solution would be to simply run the algorithm once with a forward pass
and then again with a backward pass, but this seems ineffective.

For time's sake, I might try that first.

Or maybe instead, I will try to make an implementation where either the left OR the right
value can change, but only on the first step.

With this approach, I'll have to keep track of the left value for comparison. This 
value can be swapped, but only when right_edge - left_edge = 1.

--- 
Ok - my solution above works, but it is far too slow. It takes around 9 seconds to compute
the max length of a sequence with length 10,000.

I'm going to mark this as solved, but I want to find a faster solution.

---
Stepped away for a bit and realized that the obvious solution here is to use another
hash table. It doesn't matter what *order* the letters in the current window are in.
It just matters how many letters there are in the current sequence to find the max length,
and it matters how many of the least recurring letters there are in relation to k.

So we will have a running window_map that keeps track of all of the values in the current
window. 

current_window_size, max_window_size = 1, 1
1. The right edge and left edge start in the same position. 
2. step right edge over to right, update window_map with addition
3. Check to see if the k rule is met
    - K rule definition:
        1. If there is one unique value in the window_map keys, rule met
        2. If there are more than two unique values in the window_map keys, check that
        the least commonly occurring values in k do not outnumber k. if they do, the rule
        is not met.
            - To check for least commonly occurring letter occurrences, we could just sort
            the window_map by value and subtract the lowest values[:1] from k to make sure that k isn't negative.
            We could do this by subtracting the max value of values from the current window size...?
4. if it is not, step left edge over to right, update window_map with removal
5. Update current window size and max window size

In [213]:
def character_replacement(s, k):
    if len(s) == 1:
        return 1
    
    window_map = {chr(num):0 for num in range(65, 91)}
    left_edge = 0
    max_window_size = 1
    
    for right_edge in range(len(s)):
        # add current right edge to window map
        window_map[s[right_edge]] += 1
        # right_edge += 1
        if not sum(sorted(window_map.values())[:-1]) <= k:
            window_map[s[left_edge]] -= 1
            left_edge += 1
        
        max_window_size = max(max_window_size, sum(window_map.values()))
        
    return max_window_size

s = "ABAB"
k = 0
character_replacement(s, k)

1

### Afterthought
**Stats:**
Runtime
489 ms
Beats
8.97%
Memory
13.7 MB
Beats
86.29%

So I think I didn't do well to converge on the best type of solution at the beginning of this
problem. As usual, I should've realized that all of the features of the first problem
would help me solve the second problem (i.e., should've immediately realized that I needed a hash table).

Regardless, once the idea of the hash table popped into mind, I was able to get the problem solved
pretty quickly.